In [51]:
!pip install neptune-client

     |████████████████████████████████| 276 kB 7.0 MB/s 
     |████████████████████████████████| 829 kB 55.3 MB/s 
     |████████████████████████████████| 53 kB 2.7 MB/s 
     |████████████████████████████████| 180 kB 79.8 MB/s 
     |████████████████████████████████| 131 kB 90.1 MB/s 
     |████████████████████████████████| 8.4 MB 60.8 MB/s 
     |████████████████████████████████| 79 kB 10.7 MB/s 
     |████████████████████████████████| 138 kB 96.0 MB/s 
     |████████████████████████████████| 63 kB 2.5 MB/s 
     |████████████████████████████████| 127 kB 72.1 MB/s 
     |████████████████████████████████| 67 kB 6.4 MB/s 
     |████████████████████████████████| 130 kB 95.6 MB/s 
  Created wheel for neptune-client: filename=neptune_client-0.13.3-py2.py3-none-any.whl size=481965 sha256=73013353f12271cb707bc989455ad529b9d2a038a43120e13317727f5eabdd01
  Stored in directory: /root/.cache/pip/wheels/0f/57/12/8fae2c4d320108451ca55c0c1288b69721f496062ed438d63d
  Created wheel for future: filen

In [52]:
import neptune.new as neptune

run = neptune.init(
    project="heldervj/chess-nlp",
    api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiI0OTI1MzNlYS0xNTY3LTQwZjctYjMxNC05ZDNmNGFkZTViMTEifQ==",
)  # your credentials

ImportError: ignored

# Leitura e filtro das bases

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Treinando o transformer do HuggingFace

[Tutorial](https://huggingface.co/blog/how-to-train)

## Treinando um novo tokenizer

In [2]:
linhas = ['1', '2', '3', '4', '5', '6', '7', '8']
colunas = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h']

pecas = ['N', 'B', 'R', 'Q', 'K']
variacoes = ['+', '#']

casas = []

for coluna in colunas:
    for linha in linhas:
        casas.append(coluna + linha)

# Movimentos comuns

movimentos_comuns = []
movimentos_comuns_com_indicador = []

movimentos_comuns_captura = []
movimentos_comuns_captura_com_indicador = []

for peca in pecas:
    for casa in casas:
        movimentos_comuns.append(peca + casa) # movimentos comuns

        movimentos_comuns_captura.append(peca + 'x' + casa) # movimentos com captura

        if peca in ('N', 'R'):
            for indicador in linhas + colunas:
                movimentos_comuns_com_indicador.append(peca + indicador + casa)
                movimentos_comuns_captura_com_indicador.append(peca + indicador + 'x' + casa)

# Movimentos peoes

movimentos_peoes = casas.copy()
movimentos_peoes_com_captura = []
movimentos_peoes_promocao = []

for coluna in colunas:
    for casa in casas:
        movimentos_peoes_com_captura.append(coluna + 'x' + casa)

for movimento_peao in movimentos_peoes + movimentos_peoes_com_captura:
    if movimento_peao[-1] in ('1', '8'):
        for promocao in pecas:
            if promocao != 'K':
                movimentos_peoes_promocao.append(movimento_peao + '=' + promocao)

print(f'{len(movimentos_comuns)} movimentos comuns')
print(f'{len(movimentos_comuns_com_indicador)} movimentos comuns com indicador')
print(f'{len(movimentos_comuns_captura)} movimentos comuns com captura')
print(f'{len(movimentos_comuns_captura_com_indicador)} movimentos comuns com captura com indicador')
print(f'{len(movimentos_peoes)} movimentos de peões')
print(f'{len(movimentos_peoes_com_captura)} movimentos de peões com captura')
print(f'{len(movimentos_peoes_promocao)} movimentos de peões com promoção')

roques = ['O-O-O', 'O-O']

todos_movimentos = movimentos_comuns + movimentos_comuns_captura + movimentos_comuns_captura_com_indicador +\
 movimentos_comuns_com_indicador + movimentos_peoes + movimentos_peoes_com_captura + movimentos_peoes_promocao + roques

cheques = [movimento + '+' for movimento in todos_movimentos]
mates = [movimento + '#' for movimento in todos_movimentos]

especiais = ['[UNK]', '[CLS]', '[SEP]', '[PAD]', '[MASK]']

todos_movimentos = especiais + todos_movimentos + cheques + mates

print(f'{len(todos_movimentos)} movimentos mapeados')

chess_voc = dict(zip(todos_movimentos, range(len(todos_movimentos))))

320 movimentos comuns
2048 movimentos comuns com indicador
320 movimentos comuns com captura
2048 movimentos comuns com captura com indicador
64 movimentos de peões
512 movimentos de peões com captura
576 movimentos de peões com promoção
17675 movimentos mapeados


In [3]:
# We won't need TensorFlow here
!pip uninstall -y tensorflow
# Install `transformers` from master
!pip install git+https://github.com/huggingface/transformers
!pip list | grep -E 'transformers|tokenizers'
# transformers version at notebook update --- 2.11.0
# tokenizers version at notebook update --- 0.8.0rc1

Found existing installation: tensorflow 2.7.0
Uninstalling tensorflow-2.7.0:
  Successfully uninstalled tensorflow-2.7.0
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-1_vl6stw
  Running command git clone -q https://github.com/huggingface/transformers /tmp/pip-req-build-1_vl6stw
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 3.3 MB 7.1 MB/s 
     |████████████████████████████████| 895 kB 56.0 MB/s 
     |████████████████████████████████| 61 kB 458 kB/s 
     |████████████████████████████████| 596 kB 53.1 MB/s 
  Created wheel for transformers: filename=transformers-4.13.0.dev0-py3-none-any.whl size=3320644 sha256=9cd293b80bd0d0af6773c336b342eac2816fc83c5b387adfa6cc6fc2065b4d42
  Stored in directory: /tmp/pip-ephem-wheel-cache-mnd03mz_/wheels/35/2e/a7/d819e3310040329f0f47e57c9e3e7a7338aa5e74c49acfe522
Successfully built transformers
  Attemp

In [4]:
from tokenizers import Tokenizer, normalizers, pre_tokenizers
from tokenizers.models import WordLevel
from tokenizers.pre_tokenizers import Digits, WhitespaceSplit
from tokenizers.processors import TemplateProcessing
from tokenizers.trainers import WordLevelTrainer

tokenizer = Tokenizer(WordLevel(vocab=chess_voc, unk_token="[UNK]"))

tokenizer.pre_tokenizer = WhitespaceSplit()

In [5]:
tokenizer.encode('e4 e5')

Encoding(num_tokens=2, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

In [6]:
tokenizer.encode('e4 e5 Nf3 f6 Nxe5 fxe5 Qh5+ g6 Qxe5+ Qe7').tokens

['e4', 'e5', 'Nf3', 'f6', 'Nxe5', 'fxe5', 'Qh5+', 'g6', 'Qxe5+', 'Qe7']

In [5]:
from transformers import GPT2Config, TFGPT2LMHeadModel, GPT2Tokenizer, GPT2LMHeadModel, PreTrainedTokenizerFast
# loading tokenizer from the saved model path
fast_tokenizer = PreTrainedTokenizerFast(tokenizer_object=tokenizer)

fast_tokenizer.add_special_tokens({'pad_token': '[PAD]'})

0

In [53]:
config = GPT2Config(
  vocab_size=fast_tokenizer.vocab_size,
  bos_token_id=fast_tokenizer.bos_token_id,
  eos_token_id=fast_tokenizer.eos_token_id
)
# creating the model
model = GPT2LMHeadModel(config)

model.num_parameters()
# => 57 million parameters

99416832

In [6]:
# Check that we have a GPU
!nvidia-smi

Thu Dec  9 00:33:23 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [7]:
# Check that PyTorch sees it
import torch
torch.cuda.is_available()

True

In [56]:
from torch.utils.data import Dataset
import os

class LineByLineTextDataset(Dataset):
    """
    This will be superseded by a framework-agnostic approach soon.
    """

    def __init__(self, tokenizer, file_path: str, block_size: int, initial_position: bool=True):
        if os.path.isfile(file_path) is False:
            raise ValueError(f"Input file path {file_path} not found")
        # Here, we do not cache the features, operating under the assumption
        # that we will soon use fast multithreaded tokenizers from the
        # `tokenizers` repo everywhere =)

        self.tokenizer = tokenizer
        self.block_size = block_size
        self.initial_position = initial_position

        with open(file_path, encoding="utf-8") as f:
            self.lines = [line for line in f.read().splitlines() if (len(line) > 0 and not line.isspace())]

    def __len__(self):
        return len(self.lines)

    def __getitem__(self, i):

        if self.initial_position:
            start = 'a2 a7 b2 b7 c2 c7 d2 d7 e2 e7 f2 f7 g2 g7 h2 h7 Ra1 Ra8 Nb1 Nb8 Bc1 Bc8 Qd1 Qd8 Ke1 Ke8 Bf1 Bf8 Ng1 Ng8 Rh1 Rh8 '
        else:
            start = ''

        encoding = self.tokenizer.encode(start + self.lines[i], add_special_tokens=True, truncation=True, max_length=self.block_size)

        encoding = {"input_ids": torch.tensor(encoding, dtype=torch.long)}

        return encoding

In [57]:
%%time

dataset = LineByLineTextDataset(
    tokenizer=fast_tokenizer,
    file_path="drive/MyDrive/chess_dataset/chess_files/games_train.txt",
    block_size=256,
)

# Without initial position

# dataset = LineByLineTextDataset(
#     tokenizer=fast_tokenizer,
#     file_path="drive/MyDrive/chess_dataset/chess_files/games_train.txt",
#     block_size=256,
#     initial_position=False,
# )

CPU times: user 1.49 s, sys: 1.34 s, total: 2.83 s
Wall time: 8.02 s


In [58]:
%%time

dataset_valid = LineByLineTextDataset(
    tokenizer=fast_tokenizer,
    file_path="drive/MyDrive/chess_dataset/chess_files/games_valid.txt",
    block_size=256,
)


# Without initial position

# dataset_valid = LineByLineTextDataset(
#     tokenizer=fast_tokenizer,
#     file_path="drive/MyDrive/chess_dataset/chess_files/games_valid.txt",
#     block_size=256,
#     initial_position=False,
# )

CPU times: user 170 ms, sys: 86.4 ms, total: 257 ms
Wall time: 1.3 s


In [59]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=fast_tokenizer, mlm=False
)

In [60]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./chess_model_gpt2_v0",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_gpu_train_batch_size=16,
    per_device_eval_batch_size=16,
    save_steps=10_000,
    save_total_limit=2,
    prediction_loss_only=True,
    evaluation_strategy='steps',
    eval_steps=40000,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    eval_dataset=dataset_valid,
    data_collator=data_collator,
)

In [61]:
%%time
trainer.train()

Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
***** Running training *****
  Num examples = 2246434
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 140403
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.


Step,Training Loss,Validation Loss
40000,1.737400,1.656804
80000,1.565900,1.491542
120000,1.486300,1.413581


Saving model checkpoint to ./chess_model_gpt2_v0/checkpoint-10000
Configuration saved in ./chess_model_gpt2_v0/checkpoint-10000/config.json
Model weights saved in ./chess_model_gpt2_v0/checkpoint-10000/pytorch_model.bin
Saving model checkpoint to ./chess_model_gpt2_v0/checkpoint-20000
Configuration saved in ./chess_model_gpt2_v0/checkpoint-20000/config.json
Model weights saved in ./chess_model_gpt2_v0/checkpoint-20000/pytorch_model.bin
Saving model checkpoint to ./chess_model_gpt2_v0/checkpoint-30000
Configuration saved in ./chess_model_gpt2_v0/checkpoint-30000/config.json
Model weights saved in ./chess_model_gpt2_v0/checkpoint-30000/pytorch_model.bin
Deleting older checkpoint [chess_model_gpt2_v0/checkpoint-10000] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 249621
  Batch size = 16
Saving model checkpoint to ./chess_model_gpt2_v0/checkpoint-40000
Configuration saved in ./chess_model_gpt2_v0/checkpoint-40000/config.json
Model weights saved in ./chess_mo

CPU times: user 15h 47min 48s, sys: 33min 18s, total: 16h 21min 6s
Wall time: 16h 16min 13s


TrainOutput(global_step=140403, training_loss=1.7069447675529046, metrics={'train_runtime': 58573.464, 'train_samples_per_second': 38.352, 'train_steps_per_second': 2.397, 'total_flos': 2.03888558900736e+17, 'train_loss': 1.7069447675529046, 'epoch': 1.0})

In [ ]:
# Save the model without the starts

# trainer.save_model('drive/MyDrive/gpt2_chess')

In [62]:
trainer.save_model("drive/MyDrive/gpt2_chess_with_starts_")

Saving model checkpoint to drive/MyDrive/gpt2_chess_with_starts
Configuration saved in drive/MyDrive/gpt2_chess_with_starts/config.json
Model weights saved in drive/MyDrive/gpt2_chess_with_starts/pytorch_model.bin


In [24]:
# Load model withou the starts

# model = GPT2LMHeadModel.from_pretrained('drive/MyDrive/gpt2_chess').to('cuda:0')

In [31]:
model = GPT2LMHeadModel.from_pretrained('drive/MyDrive/gpt2_chess_with_starts').to('cuda:0')

In [9]:
from transformers import top_k_top_p_filtering
import torch
from torch.nn import functional as F

class ChessGPT2Player:
    def __init__(self, model, tokenizer, device='cuda:0'):
        self.model = model
        self.tokenizer = tokenizer
        self._device = device
        self.ilegal_moves = []

    def next_move(self, preview_moves, legal_moves=[]):
        inputs = self.tokenizer.encode(preview_moves, return_tensors='pt').to(self._device)

        legal_moves_ids = [fast_tokenizer.encode(move)[0] for move in legal_moves]

        # Get logits from last layer
        last_layer_logits = self.model(inputs).logits[:, -1, :]

        legal_move = legal_moves_ids[0]
        legal_move_prob = -torch.inf

        for move in legal_moves_ids:
            if last_layer_logits[0][move].item() >= legal_move_prob:
                legal_move = move
                legal_move_prob = last_layer_logits[0][move].item()

        # mask = torch.zeros_like(last_layer_logits)
        # mask[:, legal_moves_ids] = 1.

        # last_layer_logits_legal = last_layer_logits[:, mask.long()]

        top_logits = top_k_top_p_filtering(last_layer_logits, top_k=last_layer_logits.shape[1], top_p=1.0)

        # top_logits_legal = top_k_top_p_filtering(last_layer_logits_legal, top_k=last_layer_logits_legal.shape[1], top_p=1.0)

        # Softmax the logits into probabilities
        probabilities = F.softmax(top_logits, dim=-1)

        # probabilities_legal = F.softmax(top_logits_legal, dim=-1)

        # Generate next token
        generated_next_token = torch.multinomial(probabilities, num_samples=1)
        generated = torch.cat([inputs, generated_next_token], dim=-1)

        # Generate next token legal
        # generated_next_token_legal = torch.multinomial(probabilities_legal, num_samples=1)

        generated_legal = torch.cat([inputs, torch.Tensor([legal_move]).unsqueeze(-1).to(self._device)], dim=-1)

        # Get result
        result_string = tokenizer.decode(generated.tolist()[0]).split(' ')[-1]

        # Get result
        result_string_legal = tokenizer.decode(generated_legal.long().tolist()[0]).split(' ')[-1]

        if result_string_legal != result_string:
            print(f'Ilegal move registered: {result_string}')
            self.ilegal_moves.append((preview_moves, result_string))

        return result_string_legal

In [32]:
chess_player = ChessGPT2Player(model, fast_tokenizer)

# Validando através do jogo

In [11]:
!pip install chess

     |████████████████████████████████| 147 kB 9.9 MB/s 


In [12]:
class HumanPlayer:
    def __init__(self):
        pass

    def next_move(self, preview_moves='', legal_moves=[]):

        while True:
            move = input('Digite o seu movimento: ')

            if move not in legal_moves:
                print('Movimento inválido')
            else:
                break
                
        return move

In [13]:
import random

class RandomPlayer:
    def __init__(self):
        pass

    def next_move(self, preview_moves='', legal_moves=[]):
        return random.choice(legal_moves)

In [33]:
import chess
from IPython.display import display, HTML, clear_output
import time
import random

class ChessGame:
    def __init__(self, player_white, player_black, do_first_move=False, first_moves=['e4'], trained_with_initial_positions=True):
        if trained_with_initial_positions:
            self.game_repr = 'a2 a7 b2 b7 c2 c7 d2 d7 e2 e7 f2 f7 g2 g7 h2 h7 Ra1 Ra8 Nb1 Nb8 Bc1 Bc8 Qd1 Qd8 Ke1 Ke8 Bf1 Bf8 Ng1 Ng8 Rh1 Rh8'
        else:
            self.game_repr = ''
        self.player_white = player_white
        self.player_black = player_black
        self.board = chess.Board()

        if do_first_move:
            for move in first_moves:
                self.board.push_san(move)
            
            self.game_repr += ' ' + ' '.join(first_moves)


    def start(self, visual='svg', pause=0.1):

        try:
            while not self.board.is_game_over(claim_draw=True):
                if self.board.turn == chess.WHITE:
                    san = self.player_white.next_move(self.game_repr, self.get_legal_moves())
                else:
                    san = self.player_black.next_move(self.game_repr, self.get_legal_moves())

                if self.game_repr == '':
                    self.game_repr += san
                else:
                    self.game_repr += ' ' + san

                self.board.push_san(san)
                board_stop = display_board(self.board)
                html = "<b>Move %s %s, Play '%s':</b><br/>%s" % (
                        len(self.board.move_stack), cor(self.board.turn), san, board_stop)
                if visual is not None:
                    if visual == "svg":
                        clear_output(wait=True)
                    display(HTML(html))
                    if visual == "svg":
                        time.sleep(pause)
        except KeyboardInterrupt:
            msg = "Game interrupted!"
            return (None, msg, self.board)

        result = None
        if self.board.is_checkmate():
            msg = "checkmate: " + cor(not self.board.turn) + " wins!"
            result = not self.board.turn
        elif self.board.is_stalemate():
            msg = "draw: stalemate"
        elif self.board.is_fivefold_repetition():
            msg = "draw: 5-fold repetition"
        elif self.board.is_insufficient_material():
            msg = "draw: insufficient material"
        elif self.board.can_claim_draw():
            msg = "draw: claim"
        if visual is not None:
            print(msg)
        return (result, msg, self.board)

    def get_legal_moves(self):
        legal_moves = self.board.legal_moves.__repr__()

        legal_moves = legal_moves[legal_moves.find('(')+1:legal_moves.find(')')]

        legal_moves = legal_moves.replace(' ', '').split(',')

        return legal_moves
    

def display_board(board, use_svg=True):
    if use_svg:
        return board._repr_svg_()
    else:
        return "<pre>" + str(board) + "</pre>"

def cor(jogador):
    return 'White' if jogador == chess.WHITE else 'Black'

In [39]:
# Play against computer

human_player = HumanPlayer()

chess_game = ChessGame(human_player, chess_player, trained_with_initial_positions=True)

chess_game.start()

checkmate: Black wins!


(False,
 'checkmate: Black wins!',
 Board('8/R7/6k1/p1p3q1/2P1P3/1P3npp/P6r/7K w - - 2 46'))

In [40]:
chess_game.game_repr

'a2 a7 b2 b7 c2 c7 d2 d7 e2 e7 f2 f7 g2 g7 h2 h7 Ra1 Ra8 Nb1 Nb8 Bc1 Bc8 Qd1 Qd8 Ke1 Ke8 Bf1 Bf8 Ng1 Ng8 Rh1 Rh8 e4 c5 Ne2 Nc6 d3 g6 g3 Bg7 Bg2 d6 O-O e6 Nd2 Nge7 b3 O-O c4 f5 Rb1 a5 Re1 e5 Bb2 f4 gxf4 exf4 Bxg7 Kxg7 Nxf4 Rxf4 f3 Ne5 Nf1 N7c6 Ng3 Qh4 Rb2 Nd4 Rf1 Bh3 Bxh3 Qxh3 Kh1 Raf8 Rd2 Qh4 Rg1 g5 Qf1 g4 Qd1 Qg5 Rgg2 h5 Rg1 h4 Ne2 Nxe2 Rxe2 h3 Rg3 Rxf3 Re1 Rf2 Re2 Rxe2 Qxe2 Rf3 Rg1 Rxd3 Qe1 Nf3 Qe2 Rd2 Rf1 Rxe2 Rd1 Nd4 Rf1 Nf3 Rd1 Nxh2 Rxd6 Nf3 Rd7+ Kg6 Rxb7 g3 Ra7 Rh2#'

In [1]:
# Computer agains random

random_player = RandomPlayer()

chess_game = ChessGame(chess_player, random_player, do_first_move=True)

chess_game.start()

NameError: ignored

In [72]:
chess_game.game_repr

'a2 a7 b2 b7 c2 c7 d2 d7 e2 e7 f2 f7 g2 g7 h2 h7 Ra1 Ra8 Nb1 Nb8 Bc1 Bc8 Qd1 Qd8 Ke1 Ke8 Bf1 Bf8 Ng1 Ng8 Rh1 Rh8 e4 c6 d4 Qc7 Nc3 Qe5 Nf3 Qe6 Bd3 Qxe4+ Be2 Qxc2 Qxc2 f5 O-O c5 dxc5 Na6 Be3 e6 Nb5 Nc7 Nxc7+ Kd8 Nxa8 h6 Rfd1 Nf6 c6 b6 Rac1 a6 Nxb6 Bb4 Nxc8 Bc5 Bxc5 Ng8 Bb6+ Ke8 c7 h5 Nd6+ Ke7 c8=Q Kf6 Bd8+ Kg6 Ne5+ Kh6 Bg5+ Kh7 Qxf5+ exf5 Bxh5 Ne7 Bg6+ Nxg6 Nxg6 a5 Nxh8 f4 Bxf4 g6 Nxg6 Kg7 Be5+ Kxg6 Rc7 Kh5 Rxd7 Kg6 Rg7+ Kh5 Rh7+ Kg5 Rg7+ Kh4 Rh7+ Kg5 Rg7+ Kh6 Rh7+'